In [1]:
## Reads in a folder of data and creates a frequency table for primer pairs that match the reads
## Basic, only for separate reads fastq documents

from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein
import sys
import os
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Seq import Seq
import subprocess
import sys
import csv
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

SeqFilePath = '/Users/annechen/Downloads/CRISPR Pools Data/test2'
PrimerID = '/Users/annechen/Downloads/sgRNAs for multiplexed CRISPR - Primer Sequences Only.csv'

output_folder = join(SeqFilePath, 'testout')
## Attempting to put data in separate output_folder
if not os.path.isdir(output_folder): 
    os.mkdir(output_folder)

def primerfreq(SeqFile, start_index):
    
    ############################################################################
    ## Load CSV File of Primer IDs into pandas dataframe
    
    primer = pd.read_csv(PrimerID)
    primer['Frequency'] = 0
    
    ############################################################################
    ## Make a dictionary for sequences
    ## Read in the fastq file  

    SeqDict = {}
    freq = {}

    with open(SeqFile, 'rU') as handle:
        for record in SeqIO.parse(handle, "fastq"):
            SeqDict[str(record.id)] = str(record.seq)
            #ProbeseqList.append(str(record.id)) ## Also make a list of all of the probe sequences for writing the output file.

    ############################################################################
    ##  Make Dictionary by looping through list and count the frequencies

    for record in SeqDict:
        seq = str(SeqDict[record])
        added = False
        newcode = ''
        if seq[-8:] != 'GGGGGGGG':
            for i in freq:
                if i[0:15] == seq[0:15]:
                    freq[i] = freq[i] + 1
                    added = True
                else:
                    newcode = seq[0:15]
            if added == False:
                freq[newcode] = 1

    ############################################################################
    ##  Convert previous dictionary to dataframe 

    fTable = pd.DataFrame.from_dict(freq, orient='index', columns=['Frequency'])
    fTable = fTable.sort_values('Frequency',ascending=False)
    fTable['ID'] = '-'
    
    ############################################################################
    ##  Enter matching primer ID if exists on fTable
    ##  Enter frequency of appearance on primer dataframe

    for seq_index, seq_row in fTable.iterrows():
        for prim_index, prim_row in primer.iterrows():
            primerID = primer.iloc[prim_index,1]
            #print seq_index + ' ' + primerID[start_index+0:start_index+15]
            if seq_index == primerID[start_index:start_index+15]:
                fTable.at[seq_index,'ID'] = primer.iloc[prim_index,0]
                primer.iloc[prim_index,2] = fTable.at[seq_index,'Frequency']
    ##pd.set_option('display.max_rows', 208) ##allows you to view entirety of fTable w/o truncation

    ##print (fTable)
    
    primer = primer.sort_values('Frequency',ascending=False)
    primer = primer.reset_index()
    del primer['index']
    
    ############################################################################
    ##  Export primer frequency data as csv, managing file path
    
    csv_file = join (output_folder, os.path.splitext(os.path.basename(SeqFile))[0] + '_primerMatch.csv')
    
    ##csv_file = join(output_folder, os.path.splitext(SeqFile)[0] + '_primerMatch.csv')

    primer.to_csv(csv_file, sep='\t', index=False)
    primercsv = pd.read_csv('primerMatch.csv',sep='\t')
    #print(primercsv)


for f in listdir(SeqFilePath):
    if not f.startswith('.'):
        ## In R2 file, sequencing left off first base, readjusted index to shift one right
        if '_R2_' in f:
            ##primerfreq(f,1)
            primerfreq(join(SeqFilePath,f),1)
        if '_R1_' in f:
            ##primerfreq(f,0)

            primerfreq(join(SeqFilePath, f),0)

In [15]:
## Code for csv of paired primers from a list of seperate reads

sep_reads = '/Users/annechen/Downloads/CRISPR Pools Data/test'

############################################################################
## Finding matching files in the folder to designate R1 as forward and R2 as reverse

fastq_data = []

for f1 in listdir(sep_reads):
    if '_R1_' in f1 and not f1.startswith('.'):
        forward_f = f1
        for f2 in listdir(sep_reads):
            if '_R2_' in f2 and not f2.startswith('.'):
                if f1.replace('_R1','') == f2.replace('_R2',''):
                    reverse_f = f2
                    
                    ############################################################################
                    ## Creating dataframe with ID, forward, and reverse reads
                    
                    with open (join(sep_reads,f1), 'rU') as forward:
                        for forward_data in SeqIO.parse(forward, 'fastq'):
                            fastq_data.append([str(forward_data.id), str(forward_data.seq)])
                    
                    fastq_df = pd.DataFrame(fastq_data, columns = ['read_ID', 'forward'])
                    index_pos = 0
                    
                    fastq_df['reverse'] = ''
                    with open (join(sep_reads,f2),'rU') as reverse:
                        for reverse_data in SeqIO.parse(reverse, 'fastq'):
                            rev_code = str(reverse_data.seq)
                            ##print fastq_df.at[index_pos, 'read_ID']
                            if str(reverse_data.id) == fastq_df.at[index_pos,'read_ID']:
                                fastq_df.at[index_pos, 'reverse'] = rev_code
                                index_pos += 1
                                
                    ############################################################################            
                    ## Making file name            
                    initial_file_name = os.path.basename(f1.replace('_R1',''))
                    combined_file = join(sep_reads, os.path.splitext(initial_file_name)[0] + '_combined.csv')
                    
                    ##print (initial_file_name)
                    ##print os.path.basename(initial_file_name)
                    ##os.path.splitext(os.path.basename(SeqFile))[0] 
                    ## Making csv file
                    fastq_df.to_csv(combined_file, sep='\t', index=False)
                    ##fastqcsv = pd.read_csv()
                    

1-1_S2_L001_001.fastq
1-1_S2_L001_001.fastq
36_S31_L001_001.fastq
36_S31_L001_001.fastq


In [ ]:
## Working doc that reads in a folder of combined reads and outputs data (only works on one doc at a time)

PrimerID = '/Users/annechen/Downloads/sgRNAs for multiplexed CRISPR - Primer Sequences Only.csv'
combined = '/Users/annechen/Downloads/CRISPR Pools Data/test2/results/1-1_S2_L001_001_combined.csv'
path = '/Users/annechen/Downloads/CRISPR Pools Data/test2/results'


comb_reads = pd.read_csv(combined, sep='\t')
primer = pd.read_csv(PrimerID)
primer.head()

############################################################################
## Create 6 dataframes to hold different information

concordant = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
discordant = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
onematch = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
nomatch = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
freqTable = pd.DataFrame(columns = ['Forward Prim ID', 'Reverse Prim ID', 'Frequency'])
aggData = {}

############################################################################
## Load PrimerID values into freqTable

## Need to change this part of code according to name of primer sequences
for index, row in primer.iterrows():
    if index == 30:
        break
    freqTable = freqTable.append({'Forward Prim ID':primer.iloc[index,0],'Reverse Prim ID':primer.iloc[index+30, 0], 'Frequency':0}, ignore_index=True)
                         
############################################################################
## Iterate through combined list to sort read pairs

for seq_index, seq_row in comb_reads.iterrows():
    
    fseq = comb_reads.iloc[seq_index,1]
    rseq = comb_reads.iloc[seq_index,2]
    
    ##Gfor and Grev = True if there is G-Tail at end of sequence, False if there is no G-tail
    Gfor = fseq[-8:] == 'GGGGGGGG'
    Grev = rseq[-8:] == 'GGGGGGGG'
    ##fID and rID will be changed later if a match is found and there is no G-tail
    fID = 'no forward match'
    rID = 'no reverse match'
    
    ############################################################################
    ## Match each read without a G-tail to a read ID
    
    if not Gfor or not Grev:
        for p_index, p_row in primer.iterrows():
            primerID = primer.iloc[p_index,1]

            if not Gfor and fseq[0:15] == primerID[0:15]:
                fID = primer.iloc[p_index, 0]
            if not Grev and rseq[0:15] == primerID[1:16]:
                rIDcode = primer.iloc[p_index, 0]
                ## Need to change this part of code according to name of primer sequences (manually matches rev code to forward by subtracting 30)
                rID = rIDcode[0:2] + str(int(rIDcode[2:])-30)
    
    
    ############################################################################
    ## Sort read into one of four categories depending on presence of G-Tail and PrimerID Match
    
    if Gfor and Grev:
        nomatch = pd.concat([nomatch, comb_reads.iloc[[seq_index]]])
        
    elif fID == rID:
        ## print 'concordant ' + comb_reads.iloc[seq_index, 0][-4:]
        concordant = pd.concat([concordant, comb_reads.iloc[[seq_index]]])
        
        ## Input info into concordant pair frequency table
        for index, row in freqTable.iterrows():
            if fID == freqTable.iloc[index,0]:
                freqTable.iloc[index,2] += 1
        
    elif fID != 'no forward match' and rID != 'no reverse match':
        ## print 'discordant ' + comb_reads.iloc[seq_index, 0][-4:]
        discordant = pd.concat([discordant, comb_reads.iloc[[seq_index]]])
        
    elif fID != 'no forward match' or rID != 'no reverse match':
        onematch = pd.concat([onematch, comb_reads.iloc[[seq_index]]])
        ## print 'one match ' + comb_reads.iloc[seq_index, 0][-4:]
        
    else:
        nomatch = pd.concat([nomatch, comb_reads.iloc[[seq_index]]])

## Sort freqTable for primer freq from largest to smallest
freqTable = freqTable.sort_values('Frequency', ascending=False)
freqTable = freqTable.reset_index()
del freqTable['index']

############################################################################
## Creating aggregate data dictionary and load into new dataframe

aggData['Concordant'] = len(concordant.index)
aggData['Discordant'] = len(discordant.index)
aggData['One ID Match'] = len(onematch.index)
aggData['No ID Match'] = len(nomatch.index)

aggData_df = pd.DataFrame(aggData.items(), columns=['Pair Type', 'Frequency'])

############################################################################
## Create csv files for each dataframe

concord_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_concordant_file.csv')
concordant.to_csv(concord_csv, sep='\t', index=False)

discord_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_discordant_file.csv')
discordant.to_csv(discord_csv, sep='\t', index=False)

onematch_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_single_match_file.csv')
onematch.to_csv(onematch_csv, sep='\t', index=False)

nomatch_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_no_matches_file.csv')
nomatch.to_csv(nomatch_csv, sep='\t', index=False)

freqTable_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_concordant_freqTable.csv')
freqTable.to_csv(freqTable_csv, sep='\t', index=False)

aggData_df_csv = join(path, os.path.splitext(os.path.basename(combined))[0] + '_aggregate_data.csv')
aggData_df.to_csv(aggData_df_csv, sep='\t', index=False)


In [10]:
## Reads in a folder of seperate fastq reads
## Creates output of combined reads from fastq files and 6 docs of data:
## 6 docs: list of concordant, discordant, single match, and no match reads, a frequency table of concordant reads, and an aggregate table of the number of reads of each type

from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein
import sys
import os
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Seq import Seq
import subprocess
import sys
import csv
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

sep_reads = '/Users/annechen/Downloads/CRISPR Pools Data/test2'
PrimerID = '/Users/annechen/Downloads/sgRNAs for multiplexed CRISPR - Primer Sequences Only.csv'

############################################################################
## Creating output folder to put results in

output_folder = join(sep_reads, 'results')
## Put data in separate output_folder
if not os.path.isdir(output_folder): 
    os.mkdir(output_folder)
    
############################################################################
## Create function to analyze combined reads csv file and split data into the 6 docs
                    
def pairAnalyze(CombSeqFile, PrimerIDFile):
    
    comb_reads = pd.read_csv(CombSeqFile, sep='\t')
    primer = pd.read_csv(PrimerIDFile)
    
    ############################################################################
    ## Create 6 dataframes to hold different information

    concordant = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
    discordant = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
    onematch = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
    nomatch = pd.DataFrame(columns = ['read_ID', 'forward', 'reverse'])
    freqTable = pd.DataFrame(columns = ['Forward Prim ID', 'Reverse Prim ID', 'Frequency'])
    aggData = {}
    
    ############################################################################
    ## Load PrimerID values into freqTable

    ## Need to change this part of code according to name of primer sequences
    for index, row in primer.iterrows():
        if index == 30:
            break
        freqTable = freqTable.append({'Forward Prim ID':primer.iloc[index,0],'Reverse Prim ID':primer.iloc[index+30, 0], 'Frequency':0}, ignore_index=True)

    ############################################################################
    ## Iterate through combined list to sort read pairs

    for seq_index, seq_row in comb_reads.iterrows():

        fseq = comb_reads.iloc[seq_index,1]
        rseq = comb_reads.iloc[seq_index,2]

        ##Gfor and Grev = True if there is G-Tail at end of sequence, False if there is no G-tail
        Gfor = fseq[-8:] == 'GGGGGGGG'
        Grev = rseq[-8:] == 'GGGGGGGG'
        ##fID and rID will be changed later if a match is found and there is no G-tail
        fID = 'no forward match'
        rID = 'no reverse match'
        
        ############################################################################
        ## Match each read without a G-tail to a read ID

        if not Gfor or not Grev:
            for p_index, p_row in primer.iterrows():
                primerID = primer.iloc[p_index,1]

                if not Gfor and fseq[0:15] == primerID[0:15]:
                    fID = primer.iloc[p_index, 0]
                if not Grev and rseq[0:15] == primerID[1:16]:
                    rIDcode = primer.iloc[p_index, 0]
                    ## Need to change this part of code according to name of primer sequences (manually matches rev code to forward by subtracting 30)
                    rID = rIDcode[0:2] + str(int(rIDcode[2:])-30)

        ############################################################################
        ## Sort read into one of four categories depending on presence of G-Tail and PrimerID Match

        if Gfor and Grev:
            nomatch = pd.concat([nomatch, comb_reads.iloc[[seq_index]]])

        elif fID == rID:
            ## print 'concordant ' + comb_reads.iloc[seq_index, 0][-4:]
            concordant = pd.concat([concordant, comb_reads.iloc[[seq_index]]])

            ## Input info into concordant pair frequency table
            for index, row in freqTable.iterrows():
                if fID == freqTable.iloc[index,0]:
                    freqTable.iloc[index,2] += 1
                
        elif fID != 'no forward match' and rID != 'no reverse match':
            ## print 'discordant ' + comb_reads.iloc[seq_index, 0][-4:]
            discordant = pd.concat([discordant, comb_reads.iloc[[seq_index]]])

        elif fID != 'no forward match' or rID != 'no reverse match':
            onematch = pd.concat([onematch, comb_reads.iloc[[seq_index]]])
            ## print 'one match ' + comb_reads.iloc[seq_index, 0][-4:]

        else:
            nomatch = pd.concat([nomatch, comb_reads.iloc[[seq_index]]])

    ## Sort freqTable for primer freq from largest to smallest
    freqTable = freqTable.sort_values('Frequency', ascending=False)
    freqTable = freqTable.reset_index()
    del freqTable['index']

    ############################################################################
    ## Creating aggregate data dictionary and load into new dataframe

    aggData['Concordant'] = len(concordant.index)
    aggData['Discordant'] = len(discordant.index)
    aggData['One ID Match'] = len(onematch.index)
    aggData['No ID Match'] = len(nomatch.index)

    aggData_df = pd.DataFrame(aggData.items(), columns=['Pair Type', 'Frequency'])

    ############################################################################
    ## Create csv files for each dataframe

    concord_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_concordant_file.csv')
    concordant.to_csv(concord_csv, sep='\t', index=False)

    discord_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_discordant_file.csv')
    discordant.to_csv(discord_csv, sep='\t', index=False)

    onematch_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_single_match_file.csv')
    onematch.to_csv(onematch_csv, sep='\t', index=False)

    nomatch_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_no_matches_file.csv')
    nomatch.to_csv(nomatch_csv, sep='\t', index=False)

    freqTable_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_concordant_freqTable.csv')
    freqTable.to_csv(freqTable_csv, sep='\t', index=False)

    aggData_df_csv = join(os.path.dirname(CombSeqFile), os.path.splitext(os.path.basename(CombSeqFile))[0] + '_aggregate_data.csv')
    aggData_df.to_csv(aggData_df_csv, sep='\t', index=False)

############################################################################
## Fill results folder with combined reads pooling paired separate reads together

for f1 in listdir(sep_reads):
    if '_R1_' in f1 and not f1.startswith('.'):
        forward_f = f1
        for f2 in listdir(sep_reads):
            if '_R2_' in f2 and not f2.startswith('.'):
                if f1.replace('_R1','') == f2.replace('_R2',''):
                    
                    fastq_data = []
                    reverse_f = f2
                    
                    ############################################################################
                    ## Creating dataframe with ID, forward, and reverse reads
                    
                    with open (join(sep_reads,f1), 'rU') as forward:
                        for forward_data in SeqIO.parse(forward, 'fastq'):
                            fastq_data.append([str(forward_data.id), str(forward_data.seq)])
                    
                    fastq_df = pd.DataFrame(fastq_data, columns = ['read_ID', 'forward'])
                    index_pos = 0
                    
                    fastq_df['reverse'] = ''
                    with open (join(sep_reads,f2),'rU') as reverse:
                        for reverse_data in SeqIO.parse(reverse, 'fastq'):
                            rev_code = str(reverse_data.seq)
                            ##print fastq_df.at[index_pos, 'read_ID']
                            if str(reverse_data.id) == fastq_df.at[index_pos,'read_ID']:
                                fastq_df.at[index_pos, 'reverse'] = rev_code
                                index_pos += 1
                                
                    ############################################################################            
                    ## Making file name            
                    initial_file_name = os.path.basename(f1.replace('_R1',''))
                    combined_file = join(output_folder, os.path.splitext(initial_file_name)[0] + '_combined.csv')
                    
                    ## Making csv file
                    fastq_df.to_csv(combined_file, sep='\t', index=False)

## Load CSV File of Primer IDs into pandas dataframe
##primer = pd.read_csv(PrimerID)

for f in listdir(output_folder):
    if not f.startswith('.'):
        pairAnalyze(join(output_folder, f), PrimerID)
    
    
    

In [ ]:
## Makes a csv containing all the data necessary to graph the results
## Requires a folder of all data files generated from previous cell

%matplotlib inline

import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import matplotlib

data = '/Users/annechen/Downloads/CRISPR Pools Data/results'
PrimerID = '/Users/annechen/Downloads/sgRNAs for multiplexed CRISPR - Primer Sequences Only.csv'

primer = pd.read_csv(PrimerID)
graph_data = pd.DataFrame(columns = ['Sample ID', 'Total Reads', 'Discordant', 'Single match', 'No match', 'Concordant'])
avg_rep = pd.DataFrame(columns = ['Forward ID', 'Reverse ID', 'Forward Sequence', 'Reverse Sequence', '% Frequency'])

############################################################################
## Input info from primer doc into final aggregate dataframes

for index, row in primer.iterrows():
    if index < 30:
        primerID = primer.iloc[index,1]
        pairs = primer.iloc[index,0] + ', ' + primer.iloc[index+30,0]
        graph_data[pairs] = 0
            
        avg_rep = avg_rep.append({'Forward ID':primer.iloc[index,0],'Reverse ID':primer.iloc[index+30,0], 
                                  'Forward Sequence':primer.iloc[index,1], 'Reverse Sequence':primer.iloc[index+30,1]}, 
                                 ignore_index=True)


for f in listdir(data):
    
    if '_aggregate_data.csv' in f:
        agg_data = pd.read_csv(join(data, f), sep='\t')
        sample_name = f.split('_')[0]
        total = agg_data.iloc[0,1] + agg_data.iloc[1,1] + agg_data.iloc[2,1] + agg_data.iloc[3,1]
        graph_data = graph_data.append({'Sample ID':sample_name, 'Total Reads':total, 'Discordant':agg_data.iloc[3,1],
                                       'Single match':agg_data.iloc[0,1],'No match':agg_data.iloc[2,1],
                                       'Concordant':agg_data.iloc[1,1]}, ignore_index=True)

for f in listdir(data): 
    
    if '_concordant_freqTable.csv' in f:
        freq = pd.read_csv(join(data,f), sep='\t')
        sample_name = f.split('_')[0]
        for index, row in graph_data.iterrows():
            if graph_data.iloc[index,0] == sample_name:
                for f_index, f_row in freq.iterrows():
                    primerpair =  freq.iloc[f_index,0] + ', ' + freq.iloc[f_index, 1]
                    graph_data.loc[index, primerpair] = freq.iloc[f_index, 2]        

## Optional: Add total columns to calculate primer totals to graph_data df for column and row 
## (if you only uncomment this part w/o modification, will mess up total values for calculating avg_rep)

##graph_data.loc['Column Total'] = graph_data.sum(numeric_only=True, axis = 0, skipna = True) 
##graph_data.loc[:,'Primer Count Total'] = graph_data.sum(numeric_only=True, axis = 1, skipna = True)

############################################################################
## Calculate frequency for average primer representation dataframe based on first 20 std cond experiments

std_cond = graph_data.head(20).copy()

std_cond.loc['Total'] = std_cond.sum(numeric_only=True, axis = 0, skipna = True) 
std_cond.loc[:,'Primer Count Total'] = std_cond.sum(numeric_only=True, axis = 1, skipna = True)

totalcount = std_cond.iloc[len(std_cond)-1, len(std_cond.columns)-1]

counter = 0
for i in range(6, len(std_cond.columns)-1):
    occurence = std_cond.iloc[len(std_cond)-1,i]
    frequency = (occurence/totalcount)*100
    avg_rep.iloc[counter, 4] = round(frequency, 3)
    counter += 1

############################################################################
## Load both dataframes into csv files

graph_data_csv = join(data, 'graphable_data.csv')
graph_data.to_csv(graph_data_csv, sep='\t', index=False)

avg_rep_csv = join(data, 'avg_primer_representation.csv')
avg_rep.to_csv(avg_rep_csv, sep='\t', index=False)



# Testing: Using online blast to generate custom db

Everything below this is me trying to figure out how to use BLAST through Biopython through online BLAST

In [68]:
## Test to try to blast a sequence over internet

from Bio.Blast import NCBIWWW
from Bio import SeqIO

result_handle = NCBIWWW.qblast("blastn","nt", "CTGGGACCCGACAGTTGTCAT")
with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
    
result_handle.close()
result_handle = open("my_blast.xml")

from Bio.Blast import NCBIXML

blast_record = NCBIXML.read(result_handle)

E_VALUE_THRESH = 0.5

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

Note: tested the commands through terminal
<br>** In order to run short sequences of alignment, have to use -word_size 7 or else the code will say there are no alignments even when there are
<br>** Making a blast database (Makeblastdb) is easiest done through terminal, but actual local blast can be through jupyter notebook

In [192]:
## Test to make a database from a downloaded genome off flybase

from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Blast import NCBIXML

file = '/Users/annechen/Downloads/dmel-all-chromosome-r5.49.fasta'

## Command Line args for local blast

## makeblastdb -in dmel-all-chromosome-r5.49.fasta -dbtype nucl
## blastn -query testprimerf.fasta -db dmel-all-chromosome-r5.49.fasta -evalue 0.1 -word_size 7

blastx_cline = NcbiblastxCommandline(cmd='blastn', query='PCRProject/query.fasta', db='PCRProject/dmel-all-chromosome-r5.49.fasta', evalue=0.1, out='PCRProject/TestBlast.xml', outfmt=5, word_size=7)

blastx_cline()




('', '')

In [37]:
## Make a fasta file of all primers given a avg_rep.csv (generated from cell 5)

import pandas as pd
import numpy as np

avg_rep_file = '/Users/annechen/Downloads/CRISPR Pools Data/results/avg_primer_representation.csv'

avg_rep = pd.read_csv(avg_rep_file, sep = '\t')

primerf = open("PCRProject/primerf.fasta", 'w')
for index, row in avg_rep.iterrows():
    primerf.write('>'+avg_rep.iloc[index,0]+' row='+str(index)+'\n'+avg_rep.iloc[index,2]+'\n')
    primerf.write('>'+avg_rep.iloc[index,1]+' row='+str(index)+'\n'+avg_rep.iloc[index,3]+'\n')

primerf.close()

In [27]:
## BLAST Fasta file

##In terminal: to make db from fasta
##makeblastdb -in dmel-all-chromosome-r5.49.fasta -dbtype nucl


from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Blast import NCBIXML

primerf = 'PCRProject/primerf.fasta'
dmeldb = 'PCRProject/dmel-all-aligned-r5.49.fasta'

blastx_cline = NcbiblastxCommandline(cmd='blastn', query=primerf, db=dmeldb, evalue=0.5, out='PCRProject/TestBlast.xml', outfmt=5)

blastx_cline()


('', '')

In [135]:
## Create a df of the blast xml results file
## This cell ends up not being used (use below cell to create a dictionary instead)

from Bio.Blast import NCBIXML

avg_rep_file = '/Users/annechen/Downloads/CRISPR Pools Data/results/avg_primer_representation.csv'
blastdf = pd.DataFrame(columns = ['primerID', 'rowID', 'hitID','locStart','locEnd'])

result_handle = open("PCRProject/9T6PZX5C016-Alignment.xml")
##result_handle = open("PCRProject/Test.xml")

blast_records = NCBIXML.parse(result_handle)

for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect< 0.02:
                ID = blast_record.query
                blastdf = blastdf.append({'primerID':ID.split('r')[0],'rowID':ID.split('=')[1],'hitID':alignment.hit_id,
                                         'locStart':hsp.sbjct_start, 'locEnd':hsp.sbjct_end}, ignore_index=True)
                break

                
blastdf.head()

                
                
                

,primerID,rowID,hitID,locStart,locEnd
0,JH441,0,gi|669632474|ref|NC_004354.4|,759135,759155
1,JH471,0,gi|669632474|ref|NC_004354.4|,759384,759362
2,JH442,1,gi|669632474|ref|NC_004354.4|,764953,764975
3,JH472,1,gi|669632474|ref|NC_004354.4|,765211,765192
4,JH443,2,gi|669632474|ref|NC_004354.4|,796293,796318


In [147]:
## Create a nested dict of data in the blast xml results file 
## Basically does same thing as above cell but through a different method to make easier data retrieval

from Bio.Blast import NCBIXML

avg_rep_file = '/Users/annechen/Downloads/CRISPR Pools Data/results/avg_primer_representation.csv'
blastdf = pd.DataFrame(columns = ['primerID', 'rowID', 'hitID','locStart','locEnd'])
blast_result_dict = {}

result_handle = open("PCRProject/9T6PZX5C016-Alignment.xml")
##result_handle = open("PCRProject/Test.xml")

## Create initial nested dict of data from blast xml file

blast_records = NCBIXML.parse(result_handle)
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect< 0.02:
                ID = blast_record.query
                hitId = alignment.hit_id
                rowId = ID.split('=')[1]
                rowRec = blast_result_dict.get(rowId) 
                if rowRec == None:
                    rowRec = {}
                    blast_result_dict[rowId] = rowRec
                hitList = rowRec.get(hitId)
                if hitList == None:
                    hitList = []
                    rowRec[hitId] = hitList
                hitEntry = {}
                hitEntry['primerID'] =  ID.split('r')[0]
                hitEntry['start'] = hsp.sbjct_start
                hitEntry['end'] = hsp.sbjct_end
                hitList.append (hitEntry)

                break

                
## Loop through dictionary to throw out chromosome repeats: isolate matching primer sequences

for row_num in blast_result_dict:
    hits = blast_result_dict[row_num]
    tobedeleted= []
    for locdict in hits:
        hitList = hits[locdict]
        if len(hitList) != 2:
            tobedeleted.append(locdict)
    for i in tobedeleted:
        del hits[i]

## blast_result_dict holds a nested dictionary of all the information
##print blast_result_dict

{u'24': {u'gi|669632474|ref|NC_004354.4|': [{'start': 1922815, 'end': 1922834, 'primerID': u'JH465'}, {'start': 1923088, 'end': 1923068, 'primerID': u'JH495'}]}, u'25': {u'gi|669632474|ref|NC_004354.4|': [{'start': 2041719, 'end': 2041740, 'primerID': u'JH466'}, {'start': 2041983, 'end': 2041964, 'primerID': u'JH496'}]}, u'26': {u'gi|669632474|ref|NC_004354.4|': [{'start': 2080418, 'end': 2080437, 'primerID': u'JH467'}, {'start': 2080716, 'end': 2080696, 'primerID': u'JH497'}]}, u'27': {u'gi|669632474|ref|NC_004354.4|': [{'start': 2270729, 'end': 2270749, 'primerID': u'JH468'}, {'start': 2271027, 'end': 2271008, 'primerID': u'JH498'}]}, u'20': {u'gi|669632474|ref|NC_004354.4|': [{'start': 1669560, 'end': 1669583, 'primerID': u'JH461'}, {'start': 1669894, 'end': 1669874, 'primerID': u'JH491'}]}, u'21': {u'gi|669632474|ref|NC_004354.4|': [{'start': 1695701, 'end': 1695722, 'primerID': u'JH462'}, {'start': 1696003, 'end': 1695980, 'primerID': u'JH492'}]}, u'22': {u'gi|669632474|ref|NC_004

In [173]:
import re

blastdbfinal = pd.DataFrame(columns = ['forPrimerID', 'revPrimerID', 'hitID', 'rowID', 'locStart','locEnd'])

##min = 9999999999
##max = 0
count = 0
for row_num in blast_result_dict:
    hits = blast_result_dict[row_num]
    for locdict in hits:
        min = 9999999999
        max = 0
        hitList = hits[locdict]
        p1 = hitList[0]['primerID']
        p2= hitList[1]['primerID']
        
        m = re.search('ref\|(.*)\..+\|',locdict)
        refseqid = m.group(1)
        blastdbfinal = blastdbfinal.append({'rowID':int(row_num),'hitID':refseqid, 'forPrimerID':p1,
                                    'revPrimerID':p2}, ignore_index=True)

        for read in hitList:
            #read = hitList[i]
            if read['start']<min:
                min = read['start']
            if read['start']>max:
                max = read['start']
            if read['end']<min:
                min = read['end']
            if read['end']>max:
                max = read['end']
        blastdbfinal.iloc[count,4]=min
        blastdbfinal.iloc[count,5]=max
        count += 1

blastdbfinal = blastdbfinal.sort_values('rowID', ascending=True)
blastdbfinal = blastdbfinal.reset_index()
del blastdbfinal['index']

blastdbfinal



,forPrimerID,revPrimerID,hitID,rowID,locStart,locEnd
0,JH441,JH471,NC_004354,0,759135,759384
1,JH442,JH472,NC_004354,1,764953,765211
2,JH443,JH473,NC_004354,2,796293,796587
3,JH444,JH474,NC_004354,3,801264,801530
4,JH445,JH475,NC_004354,4,805929,806216
5,JH446,JH476,NC_004354,5,934882,935163
6,JH447,JH477,NC_004354,6,940706,940955
7,JH448,JH478,NC_004354,7,1209740,1210016
8,JH449,JH479,NC_004354,8,1236325,1236574
9,JH450,JH480,NC_004354,9,1286307,1286558


In [190]:
## Takes in a downloaded file of dmel chromosome data and creates an output fasta file of the custom db
## This was a test cell that ISN'T ULTIMATELY USED: the dmel file I downloaded didn't align quite right
    ## with the code --> had to instead download the individual chromosome file off ncbi blast website

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio.Blast import NCBIXML
import csv
import pandas as pd
import numpy as np
import re
from sets import Set

dmelfasta = 'PCRProject/dmel-all-chromosome-r5.49.fasta'

uniqueRefSeqIds = Set()
for index, row in blastdbfinal.iterrows():
    uniqueRefSeqIds.add(blastdbfinal.iloc[index,2])
    
print uniqueRefSeqIds

fastaSeq = {}


with open(dmelfasta, 'rU') as handle:
    for record in SeqIO.parse(handle, "fasta"):
       ## print record.description
        m = re.search('(REFSEQ\:(.*)\,GB:\S+);',record.description)
        if m:
            ids= m.group(1)
            refSeqID = m.group(2)
            ##print refSeqID
            ##print ids
            if refSeqID in uniqueRefSeqIds:
                fastaInfo = {}
                fastaInfo['id'] = record.id
                fastaInfo['dbxref'] = ids
                fastaInfo['seq'] = str(record.seq)
                fastaSeq[refSeqID] = fastaInfo

##print fastaSeq
f = open('PCRProject/customdb2.fasta', 'w')
for index, row in blastdbfinal.iterrows():
    working = fastaSeq[blastdbfinal.iloc[index,2]]
    start = blastdbfinal.iloc[index,4]
    end = blastdbfinal.iloc[index,5]
    f.write('>'+working['id']+' loc='+str(start)+'..'+str(end)+' dbxref='+working['dbxref']+' fwdPrimer='
            +blastdbfinal.iloc[index,0]+' revPrimer='+blastdbfinal.iloc[index,1]+'\n')
    f.write(working['seq'][start-1: end-1]+'\n')
f.close()
    


Set([u'NC_004354'])


In [191]:
## Takes in a fasta file (downloaded directly off NCBI as a separate file for each chromosome)
## Creates custom db in fasta format

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio.Blast import NCBIXML
import csv
import pandas as pd
import numpy as np
import re
from sets import Set

dmelchromfasta = 'PCRProject/sequence.fasta'

## Create set of seqref IDs (catalogues chromosome number)
## Based on results of set, download corresponding chromosome numbers of NCBI website

uniqueRefSeqIds = Set()
for index, row in blastdbfinal.iterrows():
    uniqueRefSeqIds.add(blastdbfinal.iloc[index,2])
    
print uniqueRefSeqIds

## Parses downloaded fasta file of chromosome and pulls out relevant info to put in dictionary

fastaSeq = {}

with open(dmelchromfasta, 'rU') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        m = re.search('(.*)\..+',str(record.id))
        refSeqID = m.group(1)
        if refSeqID in uniqueRefSeqIds:
            fastaInfo = {}
            fastaInfo['id'] = refSeqID
            fastaInfo['header'] = record.description
            fastaInfo['seq'] = str(record.seq)
            fastaSeq[refSeqID] = fastaInfo
        ##print record.id
        ##print record.description

## Creates a new fasta file to write in the custom db and save it        

f = open('PCRProject/customdb.fasta', 'w')
for index, row in blastdbfinal.iterrows():
    working = fastaSeq[blastdbfinal.iloc[index,2]]
    start = blastdbfinal.iloc[index,4]
    end = blastdbfinal.iloc[index,5]
    f.write('>'+working['header']+' loc='+str(start)+'..'+str(end)+' fwdPrimer='
            +blastdbfinal.iloc[index,0]+' revPrimer='+blastdbfinal.iloc[index,1]+'\n')
    f.write(working['seq'][start-1: end]+'\n')
f.close()

Set([u'NC_004354'])


# Final: Put Everything Above Together and Use Local Blast!
Basically modified code to use local blast after I figured out the word_size thing (without it my code wasn't working so I did everything with online blast)

In [ ]:
#################################################################################3
## Make a fasta file called primerf.fasta for all the primers to run through local blast

import pandas as pd
import numpy as np

avg_rep_file = '/Users/annechen/Downloads/CRISPR Pools Data/results/avg_primer_representation.csv'

avg_rep = pd.read_csv(avg_rep_file, sep = '\t')

primerf = open("PCRProject/primerf.fasta", 'w')
for index, row in avg_rep.iterrows():
    primerf.write('>'+avg_rep.iloc[index,0]+' row='+str(index)+'\n'+avg_rep.iloc[index,2]+'\n')
    primerf.write('>'+avg_rep.iloc[index,1]+' row='+str(index)+'\n'+avg_rep.iloc[index,3]+'\n')

primerf.close()

## Run local blast against primerf.fasta

In [195]:
## Create a db from an input dmel genome file

from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Blast import NCBIXML

file = 'PCRProject/dmel-all-chromosome-r5.49.fasta'

#################################################################################
## First use linux for creating db from dmel genome I downloaded off flybase

## makeblastdb -in dmel-all-chromosome-r5.49.fasta -dbtype nucl

#################################################################################
## Run a local blast and output an xml file to parse the results and primer location

## Can be done either through terminal: 
## blastn -query testprimerf.fasta -db dmel-all-chromosome-r5.49.fasta -evalue 0.1 -word_size 7

## or through jupyter notebook:
blastx_cline = NcbiblastxCommandline(cmd='blastn', query='PCRProject/primerf.fasta', db='PCRProject/dmel-all-chromosome-r5.49.fasta', 
                                     evalue=0.1, out='PCRProject/BlastResults.xml', outfmt=5, word_size=7)

blastx_cline()

('', '')

## Parse Local Blast XML File Results 
First cell: Input Blast Results XML File data into a nested dictionary. Throw out any reads on different chromosomes that do not match
<br>Second cell: Determines start and end locations of each primer and inputs final data into a dataframe (blastdbfinal)

In [204]:
## Create a nested dict of data in the blast xml results file 

from Bio.Blast import NCBIXML

blastdf = pd.DataFrame(columns = ['primerID', 'rowID', 'hitID','locStart','locEnd'])
blast_result_dict = {}

result_handle = open("PCRProject/BlastResults.xml")

#################################################################################
## Create initial nested dict of data from blast xml file

blast_records = NCBIXML.parse(result_handle)
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect< 0.02:
                ID = blast_record.query
                m = re.search('REFSEQ\:(.*)\,GB:\S+;',alignment.hit_def)
                hitId = m.group(1)
                rowId = ID.split('=')[1]
                rowRec = blast_result_dict.get(rowId) 
                if rowRec == None:
                    rowRec = {}
                    blast_result_dict[rowId] = rowRec
                hitList = rowRec.get(hitId)
                if hitList == None:
                    hitList = []
                    rowRec[hitId] = hitList
                hitEntry = {}
                hitEntry['primerID'] =  ID.split('r')[0]
                hitEntry['start'] = hsp.sbjct_start
                hitEntry['end'] = hsp.sbjct_end
                hitList.append (hitEntry)

                break

                
## Loop through dictionary to throw out chromosome repeats: isolate matching primer sequences

for row_num in blast_result_dict:
    hits = blast_result_dict[row_num]
    tobedeleted= []
    for locdict in hits:
        hitList = hits[locdict]
        if len(hitList) != 2:
            tobedeleted.append(locdict)
    for i in tobedeleted:
        del hits[i]

## blast_result_dict holds a nested dictionary of all the information
## print blast_result_dict

In [205]:
## Takes in nested dictionary from previous cell and converts it to a readable dataframe
import re

blastdbfinal = pd.DataFrame(columns = ['forPrimerID', 'revPrimerID', 'hitID', 'rowID', 'locStart','locEnd'])

##min = 9999999999
##max = 0
count = 0
for row_num in blast_result_dict:
    hits = blast_result_dict[row_num]
    for locdict in hits:
        min = 9999999999
        max = 0
        hitList = hits[locdict]
        p1 = hitList[0]['primerID']
        p2= hitList[1]['primerID']
        blastdbfinal = blastdbfinal.append({'rowID':int(row_num),'hitID':refseqid, 'forPrimerID':p1,
                                    'revPrimerID':p2}, ignore_index=True)
        for read in hitList:
            #read = hitList[i]
            if read['start']<min:
                min = read['start']
            if read['start']>max:
                max = read['start']
            if read['end']<min:
                min = read['end']
            if read['end']>max:
                max = read['end']
        blastdbfinal.iloc[count,4]=min
        blastdbfinal.iloc[count,5]=max
        count += 1

blastdbfinal = blastdbfinal.sort_values('rowID', ascending=True)
blastdbfinal = blastdbfinal.reset_index()
del blastdbfinal['index']

## blastdbfinal


,forPrimerID,revPrimerID,hitID,rowID,locStart,locEnd
0,JH441,JH471,NC_004354,0,653168,653417
1,JH442,JH472,NC_004354,1,658986,659244
2,JH443,JH473,NC_004354,2,690326,690620
3,JH444,JH474,NC_004354,3,695297,695563
4,JH445,JH475,NC_004354,4,699962,700249
5,JH446,JH476,NC_004354,5,828915,829196
6,JH447,JH477,NC_004354,6,834739,834988
7,JH448,JH478,NC_004354,7,1103773,1104049
8,JH449,JH479,NC_004354,8,1130358,1130607
9,JH450,JH480,NC_004354,9,1180340,1180591


## Matches XML file results to create custom fasta db
Don't forget to use terminal to convert fasta to db at very end

In [210]:
## Takes in a downloaded file of dmel chromosome data and creates an output fasta file of the custom db

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio.Blast import NCBIXML
import csv
import pandas as pd
import numpy as np
import re
from sets import Set

dmelfasta = 'PCRProject/dmel-all-chromosome-r5.49.fasta'

uniqueRefSeqIds = Set()
for index, row in blastdbfinal.iterrows():
    uniqueRefSeqIds.add(blastdbfinal.iloc[index,2])
    
fastaSeq = {}


with open(dmelfasta, 'rU') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        m = re.search('(REFSEQ\:(.*)\,GB:\S+);',record.description)
        if m:
            ids= m.group(1)
            refSeqID = m.group(2)
            if refSeqID in uniqueRefSeqIds:
                fastaInfo = {}
                fastaInfo['id'] = record.id
                fastaInfo['dbxref'] = ids
                fastaInfo['seq'] = str(record.seq)
                fastaSeq[refSeqID] = fastaInfo

##print fastaSeq
f = open('PCRProject/customdb2.fasta', 'w')
for index, row in blastdbfinal.iterrows():
    working = fastaSeq[blastdbfinal.iloc[index,2]]
    start = blastdbfinal.iloc[index,4]
    end = blastdbfinal.iloc[index,5]
    f.write('>'+working['id']+'_'+str(index)+' loc='+str(start)+'..'+str(end)+' dbxref='+working['dbxref']+' fwdPrimer='
            +blastdbfinal.iloc[index,0]+' revPrimer='+blastdbfinal.iloc[index,1]+'\n')
    f.write(working['seq'][start-1: end]+'\n')
f.close()
    
## After creating file, use terminal to convert to db:
## makeblastdb -in customdb2.fasta -dbtype nucl